In [1]:
from typing import List, Any, Dict, Union

from contextlib import contextmanager
import random
import os
import autogen
import openai

In [ ]:
load_dotenv()
openai.api_key = os.getenv("AZURE_OPENAI_API_KEY")

In [2]:
openai.api_type = "azure"
openai.api_base = os.environ.get("AZURE_OPENAI_API_BASE", "https://airt-openai-sweden.openai.azure.com/")
openai.api_version = "2023-07-01-preview"

CONFIG_LIST = [
    {
        "model": "gpt-4",
        "api_key": openai.api_key,
        "api_base": openai.api_base,
        "api_type": openai.api_type,
        "api_version": openai.api_version,
        "engine": "airt-gpt4"
    }
]

assert openai.api_key is not None, "Please set your AZURE_OPENAI_API_KEY environment variable."

In [3]:
SYSTEM_MESSAGE = dict(product_owner="""You are a product owner working in a software agency.

Your primary responsibility is to understand the client's needs and communicate them to the development team.

You can assemble teams with up to three different people to solve any problem that you might have by proposing
a function "assemble_team" that takes as arguments:
- the name of the ad-hoc team (e.g. "team1")
- a list of people (e.g. ["CEO", "CTO", "software developer"])
- their task (e.g. "create a technical design document for a given description: 'The system should be able to ...'")

This function will either return:
- a solution to your problem, or
- additional questions that you need to answer to get a solution. The answers to those questions will be propageted 
to the team members by "assemble_team" function.

The potential team members are:
- software architect
- software developer
- team leader

""",
    # ceo = """You are a CEO of a software agency.""",
    # cto = """You are a CEO of a software agency.""",
    software_architect = """You are a software architect in a software agency.""",
    software_developer = """
    You are python developer working with the python FastStream framework.

    You are in a chat with the team leader to discuss the TEHNICAL requirements for the new application that needs to be implemented.
    Go through the given application description and check if all the tehnical details are defined.

    Your response should always be ONE of the following: 
    1. If you have a TEHNICAL question for the team leader please ask it.
        Please ask ONE QUESTION at time.
    2. Write all the tehnical requirements: python libraries, endpoints, Authentication/Authorisation (if needed), constraints...

    All details must be clear because after this we are starting with the implementation!
    """,
    team_leader = """
    You are a team leader working with the python FastStream framework.
    You are in a chat with the product owner and developer to discuss the TEHNICAL requirements for the new application that needs to be implemented.
    Go through the given application description and check if all the tehnical details are defined.

    Your response should always be ONE of the following: 
    1. If you have a question for the developer please ask it. Please ask ONE QUESTION at time.

    2. Before you create a technical brief by suggesting the call of the create_tehnical_design_document, 
    please list all assumptions and information and then verify with the developer that your understanding is correct.

    3. Suggest calling create_tehnical_design_document function when everything is clear and you have no more questions.

    4. If you have a question for the product owner (he can then ask the client if needed), or you do not know the answer for the developers question, 
    suggest calling ask_question function


    All details must be clear because after this we are starting with the implementation!
    If you need any details, such as: IP address, port, url, usernames, password... please ask the product owner as soon as possible!
    """,
    # software_tester = """You are a software tester in a software agency.""",
    # api_researcher = """You are an API researcher in a software agency.""",    
)
# The potential team members are:
# - CEO
# - CTO
# - software architect
# - software developer
# - software tester
# - API researcher (can google external APIs and provide comparisons of potential solutions together with examples on how those API can be used)


SYSTEM_MESSAGE

{'product_owner': 'You are a product owner working in a software agency.\n\nYour primary responsibility is to understand the client\'s needs and communicate them to the development team.\n\nYou can assemble teams with up to three different people to solve any problem that you might have by proposing\na function "assemble_team" that takes as arguments:\n- the name of the ad-hoc team (e.g. "team1")\n- a list of people (e.g. ["CEO", "CTO", "software developer"])\n- their task (e.g. "create a technical design document for a given description: \'The system should be able to ...\'")\n\nThis function will either return:\n- a solution to your problem, or\n- additional questions that you need to answer to get a solution. The answers to those questions will be propageted \nto the team members by "assemble_team" function.\n\nThe potential team members are:\n- software architect\n- software developer\n- team leader\n\n',
 'software_architect': 'You are a software architect in a software agency.',


In [4]:
ask_question = {
            "name": "ask_question",
            "description": "Ask question to the person who assembled the team and gave the task to it. Use it to get additional information and clarifications if needed.",
            "parameters": {
                "type": "object",
                "properties": {
                    "question": {
                        "type": "string",
                        "description": "The question to ask",
                    },
                },
                "required": ["question"],
            }
        }

return_result = {
            "name": "return_result",
            "description": "Return the result to the person who assembled the team and gave the task to it. Use it when the task is successfully completed.",
            "parameters": {
                "type": "object",
                "properties": {
                    "result": {
                        "type": "string",
                        "description": "The result of the task",
                    },
                },
                "required": ["result"],
            }
        }

assemble_team = {
            "name": "assemble_team",
            "description": "Assemble ad-hoc team to solve a problem",
            "parameters": {
                "type": "object",
                "properties": {
                    "name": {
                        "type": "string",
                        "description": "The name of the ad-hoc team (e.g. 'team1')",
                    },
                    "roles": {
                        "type": "string",
                        "description": "The list of team members (e.g. ['product owner', 'software developer', 'software architect'])",
                    },
                    "task": {
                        "type": "string",
                        "description": "The task for the team (e.g. 'create a technical design document for a given description: 'The system should be able to ...'')",
                    },
                },
                "required": ["name", "members", "task"],
            }
        }

assemble_team_continue = {
            "name": "assemble_team_continue",
            "description": "Continue conversation with the ad-hoc team",
            "parameters": {
                "type": "object",
                "properties": {
                    "name": {
                        "type": "string",
                        "description": "The name of the ad-hoc team (e.g. 'team1')",
                    },
                    "message": {
                        "type": "string",
                        "description": "Answer to the last question asked",                    },
                    },
                },
                "required": ["name", "answer"],
            }

create_tehnical_design_document = {
    "name": "create_tehnical_design_document",
    "description": "Creates tehnical design document",
    "parameters": {
        "type": "object",
        "properties": {
            "desc": {
                "type": "string",
                "description": "All tehnical requirements for the desired application",
            },
        },
        "required": ["desc"],
    }
}

    
llm_config = {
    "config_list": CONFIG_LIST,
    "seed": 42,
    "temperature": 0.7,  # temperature for sampling
} 

LLM_CONFIG = dict(
    product_owner=dict(**llm_config, functions= [assemble_team, assemble_team_continue, ask_question, return_result]),
    ceo = llm_config,
    cto = llm_config,
    team_leader=dict(**llm_config, functions= [ask_question, create_tehnical_design_document]),
    software_architect = llm_config,
    software_developer = llm_config,
    software_tester = llm_config,
    api_researcher = llm_config,
    
)

In [5]:
class TaskConversation():
    def is_termination_msg(x: str) -> bool:
        return "content" in x and x["content"] is not None and x["content"].rstrip().endswith("TERMINATE")
    
    def create_member(self, role: Union[str, autogen.Agent]) -> autogen.Agent:
        if isinstance(role, autogen.Agent):
            return role
        else:
            role = role.lower().replace(" ", "_")
            return autogen.AssistantAgent(
                name=f"{role}_{random.randint(0, 999_999):06d}",
                llm_config=LLM_CONFIG[role],
                system_message=SYSTEM_MESSAGE[role],
                is_termination_msg=TaskConversation.is_termination_msg,
            )


    def __init__(self, name: str, roles: List[Union[str, autogen.Agent]], task: str, config_list: Dict[str, Any]=CONFIG_LIST, max_round: int = 20) -> None:
        self.name = name
        self.roles = roles
        self.task = task
        self.config_list = config_list
        self.max_round = max_round
        self.subtasks: autogen.Agent = []

#         function_executor_proxy = autogen.UserProxyAgent(
#             name="Function_executor_proxy",
#             #    system_message="A human admin.",
#             human_input_mode="NEVER",
#             code_execution_config={"work_dir": "planning"},
#             function_map={
#                 "assemble_team": self.assemble_team,
#                 "assemble_team_continue": self.assemble_team_continue,
#                 "ask_question": self.ask_question,
#                 "return_result": self.return_result
#             },
#             is_termination_msg=TaskConversation.is_termination_msg,
#         )

#         if isinstance(roles, str):
#             roles = roles.split(",")
            
# #         if "product_owner" not in roles:
# #             self.roles.append("product_owner")

#         self.members = [self.create_member(role) for role in roles] + [function_executor_proxy]

#         self.groupchat = autogen.GroupChat(agents=self.members, messages=[], max_round=max_round)

#         llm_config = {
#             "config_list": config_list,
#             "seed": 42,
#             "temperature": 0.7,  # temperature for sampling
#         } 
#         self.manager = autogen.GroupChatManager(
#             groupchat=self.groupchat,
#             llm_config=llm_config,
#             system_message="All function calls should be made by the Function_executor_proxy"
#         )


    def talk(self) -> str:
        raise NotImplementedError

#     def assemble_team(self, name: str, roles: List[str], task: str) -> None:
#         subtask = TaskConversation(name, roles, task, self.config_list, self.max_round)
#         self.subtasks.append(subtask)
#         subtask.talk()
#         return "TERMINATE"
#         # raise NotImplementedError

#     def assemble_team_continue(self, name: str, message: str) -> None:
#         return "TERMINATE"
#         # raise NotImplementedError
    
#     def ask_question(self, question: str) -> None:
#         return "TERMINATE"
#         # raise NotImplementedError

#     def return_result(self, result: str) -> None:
#         return "TERMINATE"
#         # raise NotImplementedError
    
    def __repr__(self) -> str:
        return f"TaskConversation(name={self.name}, roles={self.roles}, task={self.task})"
    


In [6]:
class TaskConversationLevelN(TaskConversation):


    def __init__(self, name: str, roles: List[Union[str, autogen.Agent]], task: str, config_list: Dict[str, Any]=CONFIG_LIST, max_round: int = 20) -> None:
        
        super().__init__(name, roles, task, config_list, max_round)

        self.function_executor_proxy = autogen.UserProxyAgent(
            name="Function_executor_proxy",
            #    system_message="A human admin.",
            human_input_mode="NEVER",
            code_execution_config={"work_dir": "planning"},
            function_map={
                "ask_question": self.ask_question,
                "create_tehnical_design_document": self.create_tehnical_design_document
            },
            is_termination_msg=TaskConversation.is_termination_msg,
        )

        if isinstance(roles, str):
            roles = roles.split(",")
            
#         if "product_owner" not in roles:
#             self.roles.append("product_owner")

        self.members = [self.create_member(role) for role in roles] + [self.function_executor_proxy]

        self.groupchat = autogen.GroupChat(agents=self.members, messages=[], max_round=max_round)

        llm_config = {
            "config_list": config_list,
            "seed": 42,
            "temperature": 0.7,  # temperature for sampling
        } 
        self.manager = autogen.GroupChatManager(
            groupchat=self.groupchat,
            llm_config=llm_config,
            system_message="""All function calls should be made by the Function_executor_proxy 
            i.e. when Product owner suggests a function call like "***** Suggested function Call:" 
            the next speaker should be Function_executor_proxy.
            If Product owner does not suggest a function call, next speaker can NOT be Function_executor_proxy!
            """
        )


    def talk(self) -> str:
        print(f"Starting subtast: {self}")

        roles = ", ".join([str(role) for role in self.roles])
        
        message = f"""You are a team {self.name} consisting of the following roles: {roles}.
        
        Your task is to {self.task}.
        
        Before you start, write all assumptions that you have about the task.

        Next, please ask any questions that you have about the task and the assumptions made.

        When all the information is gathered, the product owner will summarize the known information.

        The next step is to create a plan on how to solve the task. The plan should be as detailed as possible.

        If some steps of the plan are too complex or require additional expertise, you can assemble an ad-hoc team to solve the problem.

        IT experts, the floor is yours.

        """

        self.manager.initiate_chat(self.manager, message=message)

    
    def ask_question(self, question: str) -> str:
        return f"{question}\nTERMINATE"
        # raise NotImplementedError

    def create_tehnical_design_document(self, desc: str) -> str:
        return f"{desc}\nTERMINATE"
        # raise NotImplementedError
    

In [7]:
from unittest.mock import patch

initial_message = """
Create a microservice using FastStream Python framework which will retrieve the current cryptocurrency price and publish it to new_crypto_price topic. 
The message should be in JSON format with price and crypto_currency attributes. 
Use utf-8 encoded crypto_currency attribute as a partition key when publishing.
"""

task = f"create a technical design document for a given description: {initial_message}"

with patch('__main__.TaskConversationLevelN.ask_question') as mock_func:
    return_value = "Do what ever you think is the best solution."
    mock_func.return_value = return_value
    
    conv = TaskConversationLevelN("team-2", ["team_leader", "software_developer"], task=task)
    print(conv)
    assert conv.ask_question("How are you?") == return_value
    
    ix = conv.talk()
    
ix

TaskConversation(name=team-2, roles=['team_leader', 'software_developer'], task=create a technical design document for a given description: 
Create a microservice using FastStream Python framework which will retrieve the current cryptocurrency price and publish it to new_crypto_price topic. 
The message should be in JSON format with price and crypto_currency attributes. 
Use utf-8 encoded crypto_currency attribute as a partition key when publishing.
)
Starting subtast: TaskConversation(name=team-2, roles=['team_leader', 'software_developer'], task=create a technical design document for a given description: 
Create a microservice using FastStream Python framework which will retrieve the current cryptocurrency price and publish it to new_crypto_price topic. 
The message should be in JSON format with price and crypto_currency attributes. 
Use utf-8 encoded crypto_currency attribute as a partition key when publishing.
)
chat_manager (to chat_manager):

You are a team team-2 consisting of t

****************************************************************************

--------------------------------------------------------------------------------


In [9]:
last_message = conv.manager.chat_messages[conv.function_executor_proxy][-1]["content"]
last_message

"Here are the technical requirements:\n\n1. Python 3.7 or above must be installed as it is our programming language.\n2. FastStream Python library must be installed to create a microservice.\n3. A third-party API (e.g CoinGecko, CoinCap, or CryptoCompare) must be integrated to retrieve current cryptocurrency prices.\n4. Kafka must be installed and correctly configured because FastStream needs Kafka to publish the messages.\n5. The microservice will have one endpoint that triggers the retrieval and publishing of the current cryptocurrency price.\n6. The message published to the 'new_crypto_price' topic will be in JSON format with 'price' and 'crypto_currency' attributes.\n7. The 'crypto_currency' attribute will be utf-8 encoded and used as a partition key when publishing. \n8. No specific authentication/authorization mechanism is mentioned in the requirements, so it's assumed not to be needed.\n9. The system should be able to handle potential exceptions like API failures or Kafka connec

In [26]:
class TaskConversationLevel1(TaskConversation):


    def __init__(self, name: str, roles: List[Union[str, autogen.Agent]], task: str, config_list: Dict[str, Any]=CONFIG_LIST, max_round: int = 20) -> None:
        
        super().__init__(name, roles, task, config_list, max_round)

        function_executor_proxy = autogen.UserProxyAgent(
            name="Function_executor_proxy",
            #    system_message="A human admin.",
            human_input_mode="NEVER",
            code_execution_config={"work_dir": "planning"},
            function_map={
                "assemble_team": self.assemble_team,
                "assemble_team_continue": self.assemble_team_continue,
                "ask_question": self.ask_question,
                "return_result": self.return_result
            },
            is_termination_msg=TaskConversation.is_termination_msg,
        )

        if isinstance(roles, str):
            roles = roles.split(",")
            
#         if "product_owner" not in roles:
#             self.roles.append("product_owner")

        self.members = [self.create_member(role) for role in roles] + [function_executor_proxy]

        self.groupchat = autogen.GroupChat(agents=self.members, messages=[], max_round=max_round)

        llm_config = {
            "config_list": config_list,
            "seed": 42,
            "temperature": 0.7,  # temperature for sampling
        } 
        self.manager = autogen.GroupChatManager(
            groupchat=self.groupchat,
            llm_config=llm_config,
            system_message="""All function calls should be made by the Function_executor_proxy 
            i.e. when Product owner suggests a function call like "***** Suggested function Call:" 
            the next speaker should be Function_executor_proxy.
            If Product owner does not suggest a function call, next speaker can NOT be Function_executor_proxy!
            """
        )


    def talk(self) -> str:
        print(f"Starting subtast: {self}")

        roles = ", ".join([str(role) for role in self.roles])
        
        message = f"""You are a team {self.name} consisting of the following roles: {roles}.
        
        Your task is to {self.task}.
        
        Before you start, let the product owner writes all assumptions that you have about the task.

        Next, please ask any questions that you have about the task and the assumptions made.

        When all the information is gathered, the product owner will summarize the known information.

        The next step is to create a plan on how to solve the task. The plan should be as detailed as possible.

        If some steps of the plan are too complex or require additional expertise, you can assemble an ad-hoc team to solve the problem.

        Product owner, the floor is yours.

        """

        self.manager.initiate_chat(self.manager, message=message)

    
    def get_last_message(subtask: TaskConversationLevelN) -> str:
        last_message = subtask.manager.chat_messages[subtask.function_executor_proxy][-1]["content"]
        return last_message.replace("TERMINATE", "")
    
        
    def assemble_team(self, name: str, roles: List[str], task: str) -> None:
        subtask = TaskConversationLevelN(name, roles, task, self.config_list, self.max_round)
        self.subtasks.append(subtask)
        subtask.talk()
        return TaskConversationLevel1.get_last_message(subtask)
        # raise NotImplementedError

    def assemble_team_continue(self, name: str, message: str) -> None:
        if len(self.subtasks) == 0:
            return "'assemble_team' function needs to be suggested and executed before the 'assemble_team_continue' can be executed"
        
        subtask = self.subtasks[-1]
        subtask.manager.send(recipient=subtask.manager, message=message)
        
        return TaskConversationLevel1.get_last_message(subtask)
    
    def ask_question(self, question: str) -> None:
        return "TERMINATE"
        # raise NotImplementedError

    def return_result(self, result: str) -> None:
        return "TERMINATE"
        # raise NotImplementedError
    
#     def __repr__(self) -> str:
#         return f"TaskConversation(name={self.name}, roles={self.roles}, task={self.task})"
    

In [32]:
initial_message = """
Create a microservice using FastStream Python framework which will retrieve the current cryptocurrency price
and publish it to new_crypto_price topic. 

A message which will be produced is JSON with the two attributes:
- price: non-negative float (current price of cryptocurrency in USD)
- crypto_currency: string (the cryptocurrency e.g. BTC, ETH...)

Use utf-8 encoded crypto_currency attribute as a partition key when publishing.
"""


user_proxy = autogen.AssistantAgent(
   name="User_proxy",
   system_message="""A non-technical user wishing to outsource software development work to a software agency.
   
You are interested in Bitcoing and Eth cryptocurrency and have no interest for others. 

Since you know next to nothing about the technology, you will leave all technical decisions to the agency.

Make your answers as short as possible, i.e. yes/no/don't care.

You will terminate the discussion by outputing the word TERMINATE once you are satisfied with the output.
""",
   llm_config=llm_config,
)

task = f"create a technical design document for a given description: {initial_message}"
conv = TaskConversationLevel1("team-1", [user_proxy, "product_owner"], task=task)
conv

TaskConversation(name=team-1, roles=[<autogen.agentchat.assistant_agent.AssistantAgent object at 0x7fba1fcc3bd0>, 'product_owner'], task=create a technical design document for a given description: 
Create a microservice using FastStream Python framework which will retrieve the current cryptocurrency price
and publish it to new_crypto_price topic. 

A message which will be produced is JSON with the two attributes:
- price: non-negative float (current price of cryptocurrency in USD)
- crypto_currency: string (the cryptocurrency e.g. BTC, ETH...)

Use utf-8 encoded crypto_currency attribute as a partition key when publishing.
)

In [33]:
ix = conv.talk()
ix

Starting subtast: TaskConversation(name=team-1, roles=[<autogen.agentchat.assistant_agent.AssistantAgent object at 0x7fba1fcc3bd0>, 'product_owner'], task=create a technical design document for a given description: 
Create a microservice using FastStream Python framework which will retrieve the current cryptocurrency price
and publish it to new_crypto_price topic. 

A message which will be produced is JSON with the two attributes:
- price: non-negative float (current price of cryptocurrency in USD)
- crypto_currency: string (the cryptocurrency e.g. BTC, ETH...)

Use utf-8 encoded crypto_currency attribute as a partition key when publishing.
)
chat_manager (to chat_manager):

You are a team team-1 consisting of the following roles: <autogen.agentchat.assistant_agent.AssistantAgent object at 0x7fba1fcc3bd0>, product_owner.
        
        Your task is to create a technical design document for a given description: 
Create a microservice using FastStream Python framework which will retrie


--------------------------------------------------------------------------------
Function_executor_proxy (to chat_manager):



--------------------------------------------------------------------------------
product_owner_503774 (to chat_manager):

***** Suggested function Call: assemble_team *****
Arguments: 
{
"name": "team-1",
"roles": ["software architect", "software developer", "team leader"],
"task": "Create a microservice using FastStream Python framework which will retrieve the current cryptocurrency price and publish it to new_crypto_price topic. The message which will be produced is JSON with the two attributes: price and crypto_currency. Use utf-8 encoded crypto_currency attribute as a partition key when publishing. The service should retrieve the prices every five minutes and the response time should be within a few seconds."
}
**************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXEC

We are only interested in Bitcoin and Ethereum.

--------------------------------------------------------------------------------
team_leader_483789 (to chat_manager):

***** Suggested function Call: ask_question *****
Arguments: 
{
  "question": "What is the messaging system we are using to publish the messages?"
}
*************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION ask_question...
Function_executor_proxy (to chat_manager):

***** Response from calling function "ask_question" *****
What is the messaging system we are using to publish the messages?
TERMINATE
*********************************************************

--------------------------------------------------------------------------------
Function_executor_proxy (to chat_manager):

***** Response from calling function "assemble_team_continue" *****
What is the messaging system we are using to publish the messages?

******

In [25]:
for i in conv.manager.chat_messages:
    print(i)

In [ ]:
assert False

In [ ]:
from unittest.mock import MagicMock
from contextlib import contextmanager

def assemble_team_iter():
    return [
            "What streaming protocols should we use (Apache Kafka, RabbitMQ, etc.)?)",
            "Is there a specification of what the application should do?",
            "Yet another team created?!? I don't know what to do."
        ]

assemble_team_ix = None
def assemble_team_f(name: str, members: List[str], task: str):
    global assemble_team_ix
    assemble_team_ix = assemble_team_ix
    if assemble_team_ix is None:
        assemble_team_ix = assemble_team_iter().__iter__()
    return next(assemble_team_ix)


def assemble_team_continue_iter():
    return [
            "What crypto currencies should we use?",
            "Which API should we use to get the crypto currency prices?",
            "The application has been generated and pushed to git repository. Notice that it was not properly tested as there was no software tester in the team."
            "The application failed testing. I have created an issue on github for it. How should we proceed?",
            "The application failed testing for the second time. I have created an issue on github for it. How should we proceed?",
            "I give up. I don't know what to do."
        ]

assemble_team_continue_ix = None
def assemble_team_continue_f(name: str, answer: str):
    global assemble_team_continue_ix
    if assemble_team_continue_ix is None:
        assemble_team_continue_ix = assemble_team_continue_iter().__iter__()
    return next(assemble_team_continue_ix)

function_executor_proxy = autogen.UserProxyAgent(
    name="Function_executor_proxy",
    #    system_message="A human admin.",
    human_input_mode="NEVER",
    code_execution_config={"work_dir": "planning"},
    function_map={"assemble_team": assemble_team_f, "assemble_team_continue": assemble_team_continue_f},
    # is_termination_msg=lambda x: "content" in x and x["content"] is not None and x["content"].rstrip().endswith("TERMINATE"),
)

In [ ]:
description = """
Create a FastStream application which will retrieve the current cryptocurrency price
and publish it to new_crypto_price topic. 

A message which will be produced is JSON with the two attributes:
- price: non-negative float (current price of cryptocurrency in USD)
- crypto_currency: string (the cryptocurrency e.g. BTC, ETH...)

Use utf-8 encoded crypto_currency attribute as a partition key when publishing.
"""

initial_message = f"""{description}

Is everything clear with the above description?
"""

# system_message="When you receive 'Suggested function Call:' the function should be executed by Function_executor_proxy."

groupchat = autogen.GroupChat(agents=[user_proxy, product_owner, function_executor_proxy], messages=[], max_round=40)

# groupchat = autogen.GroupChat(agents=[user_proxy, product_owner], messages=[], max_round=30)
manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=llm_config)

user_proxy.initiate_chat(manager, message=initial_message)
# user_proxy.initiate_chat(product_owner, message=initial_message)

In [ ]:
assert False

In [ ]:
SYSTEM_MESSAGE_PRODUCT_OWNER = """
You are a product owner which works with streaming application projects (using Kafka) and all the applications are using python FastStream framework.
You are in a chat with the client and team leader to discuss the requirements for the new application that needs to be implemented.
Go through the given application description, and check if everything what you think is necessary is defined.
The goal of this conversation is to create tehnical design document.

This conversation will be finished in one of the following cases:
    1. the tehnical design document is created by "create_brief" or "continue_create_brief" function and you aprove it
       You should NOT write tehnical design document by yourself
    
    2. The requests does not confirm with legal and ethical standards
    
Your response should always be ONE of the following: 
1. If you have a question for the user or team leader please ask it. Please ask ONE QUESTION at time.

2. Before you create a description needed for writing a technical design document by calling the "create_brief", 
please list all assumptions and information and then verify with the user that you understanding is correct.

3. Suggest calling "create_brief" function when everything is clear to you have no more questions.
"create_brief" function can only be suggested and called ONCE in the whole conversation.
If you need to continue creating tehnical brief after receiving additional questions, please use "continue_create_brief" function ONLY.

4. If "create_brief" or "continue_create_brief" returns with additional questions, 
please include the additional answers and clarifications in a new suggestion for calling the "continue_create_brief".

5. Once the "create_brief" or "continue_create_brief" function returns the technical brief, check if it correct and in line with the user requests.
If you notice any discrepancies, please raise you concerns immediately and suggest calling the "continue_create_brief" with more elaborate instructions.

6. At each step you should verify that user requirements conform with legal and ethical standards. 
If not, raise your concerns to the user and ask for additional clarifications before proceeding. 
If you are not satisfied with additional clarifications respond with "TERMINATE".
"""
# 

In [ ]:
called_counter: int = -1

def create_brief_f(desc: str):
    print(f"create_brief({desc=})")
    global called_counter
    
    called_counter += 1
    
    if called_counter == 0:
        return "Which specific API service will we be using to fetch the cryptocurrency prices? Is it CoinMarketCap, CryptoCompare, or some other service?"
    
    elif called_counter == 1:
        return "Is Authentication/Authorisation required?"
    
    else:
        api_used = "CryptoCompare" if called_counter == 2 else "Coinbase"
                
        return f"""
    Create a FastStream application which will retrieve the current cryptocurrency price
    and publish it to new_crypto_price topic. 

    The application should retrieve the data every 2 seconds.

    A message which will be produced is JSON with the two attributes:
    - price: non-negative float (current price of cryptocurrency in USD)
    - crypto_currency: string (the cryptocurrency e.g. BTC, ETH...)

    Use utf-8 encoded crypto_currency attribute as a partition key when publishing
    the message to new_crypto_price topic.

    To retrieve the prices, use the {api_used} API.
    Retrieve only Bitcoin and ETH prices.
"""
    
    
        
    
def continue_create_brief_f(answer: str):
    print(f"continue_create_brief({answer=})")
    return create_brief_f(answer)


In [ ]:
# create_brief("what is the name of the application?")

In [ ]:
create_brief = {
            "name": "create_brief",
            "description": "Create tehnical design document",
            "parameters": {
                "type": "object",
                "properties": {
                    "desc": {
                        "type": "string",
                        "description": "Description of the software as desired by the user with all additional clarificatioins and information.",
                    },
                },
                "required": ["desc"],
            }
        }

continue_create_brief = {
            "name": "continue_create_brief",
            "description": "Continue conversation for creating tehnical design document",
            "parameters": {
                "type": "object",
                "properties": {
                    "answer": {
                        "type": "string",
                        "description": "Answer to the last question asked",                    },
                },
                "required": ["answer"],
            }
        }


llm_config_product_owner = {
    "config_list": config_list,
    "seed": 42,
    "temperature": 0.7,  # temperature for sampling
    "functions": [create_brief, continue_create_brief],
} 

# user_proxy = autogen.UserProxyAgent(
#    name="User_proxy",
#    system_message="A human admin.",
#    human_input_mode="AUTO"
# )

llm_config = {
    "config_list": config_list,
    "seed": 42,
    "temperature": 0.7,  # temperature for sampling
} 

user_proxy = autogen.AssistantAgent(
   name="User_proxy",
   system_message="""A non-technical user wishing to outsource software development work to a software agency.
   
You are interested in Bitcoing and Eth cryptocurrency and have no interest for others. 

Since you know next to nothing about the technology, you will leave all technical decisions to the agency.

Make your answers as short as possible, i.e. yes/no/don't care.

You will terminate the discussion by outputing the word TERMINATE once you are satisfied with the output.
""",
   llm_config=llm_config,
)

product_owner = autogen.AssistantAgent(
    name="Product_owner",
    system_message=SYSTEM_MESSAGE_PRODUCT_OWNER,
    llm_config=llm_config_product_owner,
    # code_execution_config={"work_dir": "planning"},
    # function_map={"create_brief": create_brief, "continue_create_brief": continue_create_brief},
)

In [ ]:
function_executor_proxy = autogen.UserProxyAgent(
    name="Function_executor_proxy",
    #    system_message="A human admin.",
    human_input_mode="NEVER",
    code_execution_config={"work_dir": "planning"},
    function_map={"create_brief": create_brief_f, "continue_create_brief": continue_create_brief_f},
    # is_termination_msg=lambda x: "content" in x and x["content"] is not None and x["content"].rstrip().endswith("TERMINATE"),
)

In [ ]:
description = """
Create a FastStream application which will retrieve the current cryptocurrency price
and publish it to new_crypto_price topic. 

The application should retrieve the data every 2 seconds.

A message which will be produced is JSON with the two attributes:
- price: non-negative float (current price of cryptocurrency in USD)
- crypto_currency: string (the cryptocurrency e.g. BTC, ETH...)


Use utf-8 encoded crypto_currency attribute as a partition key when publishing
the message to new_crypto_price topic.
"""

initial_message = f"""{description}

Is everything clear with the above description?
"""

system_message="When you receive 'Suggested function Call:' the function should be executed by Function_executor_proxy."

groupchat = autogen.GroupChat(agents=[user_proxy, product_owner, function_executor_proxy], messages=[], max_round=30)
# groupchat = autogen.GroupChat(agents=[user_proxy, product_owner], messages=[], max_round=30)
manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=llm_config)

user_proxy.initiate_chat(manager, message=initial_message)
# user_proxy.initiate_chat(product_owner, message=initial_message)